In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data.utils import read_embeddings_dataframe

In [117]:
import os
import pandas as pd
df = read_embeddings_dataframe(os.path.join("..", "datasets", "embeddings.csv"))
# df = pd.read_pickle("embeddings_30_20.pkl.gz")
# df = pd.read_pickle("embeddings_50_20.pkl.gz")

In [118]:
type(df['embedding'].iloc[0])

numpy.ndarray

In [119]:
from src.models.clustering import perform_clusterings, create_clustering_files

In [120]:
results = perform_clusterings(df, 5, 5, 0.8)
create_clustering_files(results, ".", "clusters", "cluster_models")

In [121]:
from src.models.dimensionality_reduction import create_umap_data_and_model
from umap import UMAP

create_umap_data_and_model(df, ".", "graph_umap", "umap")

In [122]:
import plotly.express as px
import pandas as pd

users = pd.read_csv("../datasets/accounts_processed.csv", index_col=0)
users['merge'] = users['username'].str.lower()
old_index = users[users['name'] == "Zygmunt Frankiewicz"].index[1]
users = users.drop(old_index).reset_index()


In [126]:
import pandas as pd
umap_df = pd.read_csv("graph_umap.csv")
cluster_info = pd.read_csv("clusters.csv")
whole_df = pd.merge(umap_df, cluster_info, on='username')
whole_df['merge'] = whole_df['username'].str.lower()
whole_df = pd.merge(whole_df, users, on="merge").drop(columns=["username_y"]).rename(columns={"username_x": "username"})

whole_df.columns
whole_df['kmeans_cluster'] = whole_df['kmeans_cluster'].astype(str)
whole_df['gmm_cluster'] = whole_df['gmm_cluster'].astype(str)
whole_df['mean_shift_cluster'] = whole_df['mean_shift_cluster'].astype(str)

fig = px.scatter(whole_df, x='2D_x', y='2D_y', color='mean_shift_cluster', hover_data=['username', 'pozycja', 'coalition', 'party', 'name'])
fig.show(renderer='browser')

In [ ]:
fig = px.scatter_3d(whole_df, x='3D_x', y='3D_z', z='3D_y', color='coalition', hover_data=['username', 'pozycja', 'coalition', 'party', 'name'])
fig.show(renderer='browser')

In [ ]:
from src.models.clustering import cluster_db_scan
data = whole_df[['3D_x', '3D_y', '3D_z']].to_numpy()
model, labels = cluster_db_scan(data, 0.4)
whole_df['super_dbscan'] = labels
whole_df['super_dbscan'] = whole_df['super_dbscan'].astype(str)
fig = px.scatter_3d(whole_df, x='3D_x', y='3D_y', z='3D_z', color='super_dbscan', hover_data=['username'])
fig.show(renderer='browser')

In [ ]:
from src.models.dimensionality_reduction import perform_umap
import numpy as np
data = np.array(df["embedding"].tolist())
umap_10_model, umap_10_reduced = perform_umap(data, 10)

In [ ]:
_, umap_10_dbscan = cluster_db_scan(umap_10_reduced, 0.3)
whole_df['umap_10'] = umap_10_dbscan
whole_df['umap_10'] = whole_df['umap_10'].astype(str)
fig = px.scatter_3d(whole_df, x='3D_x', y='3D_y', z='3D_z', color='umap_10', hover_data=['username'])
fig.show(renderer='browser')

In [ ]:
data.shape

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances 

distances = euclidean_distances(data)

In [ ]:
distances = np.reshape(distances, -1)
distances.shape

In [ ]:
fig = px.histogram(distances)
fig.show(renderer='browser')

In [112]:
df = pd.read_pickle("../datasets/tweets_cleaned.pkl.gz")

In [ ]:
sort = df.sort_values(by="tweet_length")

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


sort[["id", "tweet", "tweet_length"]].where((sort["tweet_length"] >= 30) & (sort["tweet_length"] < 40)).dropna()

In [ ]:
fig = px.histogram(sort, x="tweet_length")
fig.show(renderer='browser')

In [ ]:
sort.where((sort["tweet_length"] >= 30) & (sort["tweet_length"] < 40)).dropna()

In [ ]:
df.where(df["tweet_length"] >= 30).dropna(how='all')["username"]

In [100]:
def generate_embeddings(tweets_dataframe: pd.DataFrame, min_tweet_length: int, min_tweets_number: int):
    tweet_data = tweets_dataframe[["id", "username", "tweet", "tweet_length"]]
    long_enough_tweets = tweet_data[tweet_data["tweet_length"] >= min_tweet_length]
    tweets_by_username_count = long_enough_tweets.groupby(by="username")[["id"]].count()
    users_to_drop = (
        tweets_by_username_count[tweets_by_username_count["id"] < min_tweets_number]
        .reset_index()["username"]
        .to_list()
    )
    df: pd.DataFrame = long_enough_tweets[~long_enough_tweets["username"].isin(users_to_drop)]  # type: ignore

    os.mkdir(f"embeddings_{min_tweet_length}_{min_tweets_number}")

    unique_usernames = df["username"].unique().tolist()


    usernames = []
    embeddings = []

    for user in unique_usernames:
        all_user_tweet_embeddings = pd.read_pickle(f"../datasets/embeddings/{user}.pkl.gz")
        selected = pd.merge(long_enough_tweets, all_user_tweet_embeddings, left_on="id", right_on="tweet_id")[["tweet_id", "username_x", "tweet_embedding"]].rename(columns={"username_x": "username"})
        mean_embedding = np.mean(np.array(selected["tweet_embedding"].tolist()), axis=0)
        usernames.append(user)
        embeddings.append(mean_embedding)
        selected.to_pickle(f"embeddings_{min_tweet_length}_{min_tweets_number}/{user}.pkl.gz")
    
    user_embeddings = pd.DataFrame({"username": usernames, "embedding": embeddings})
    user_embeddings.to_pickle(f"embeddings_{min_tweet_length}_{min_tweets_number}.pkl.gz")



In [113]:
import numpy as np
generate_embeddings(df, 50, 20)


# tweet_data = df[["id", "username", "tweet", "tweet_length"]]
# long_enough_tweets = tweet_data[tweet_data["tweet_length"] >= 30]
# all_user_tweet_embeddings = pd.read_pickle(f"../datasets/embeddings/achybicka.pkl.gz")
